In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
dataset_url = "https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz"
source_dir = '/root/.keras/datasets/images/'  # images to process
save_dir = '/drive/colab/sr/images-cropped-196/'  # processed images

In [3]:
keras.utils.get_file(origin=dataset_url, fname="oxford-iiit-pets", untar=True)

'/root/.keras/datasets/oxford-iiit-pets'

In [4]:
list_ds = tf.data.Dataset.list_files(source_dir + '*.jpg', shuffle=False)

In [12]:
import os


# randomly crop images from the dataset
def random_crop(img, crop_x, crop_y):
    img_x, img_y, _ = img.shape

    # skip smaller images
    if img_x <= crop_x or img_y <= crop_y:
        return

    max_x = img_x - crop_x
    max_y = img_y - crop_y

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    cropped_img = img[x:x + crop_x, y:y + crop_y, :]
    return cropped_img


def save_img(img, title):
    try:
        keras.utils.save_img(save_dir + title, img)
    except Exception:
        print(title)  # print offenders
        raise Exception


def crop_image(filename):
    # get the image title (name.ext)
    parts = tf.strings.split(filename, os.sep)
    title = bytes.decode(parts[-1].numpy())

    # skip non .jpg files
    if title.split('.')[-1] != 'jpg':
        return

    # skip files that have already been processed
    if os.path.isfile(source_dir + title):
        return

    try:
        img = tf.io.read_file(filename)
        img = tf.io.decode_jpeg(img)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = random_crop(img, 196, 196)
    except Exception:
        print(title)   # print offenders
        raise Exception

    # if cropping failed because of img being too small
    if img == None:
        return

    save_img(img, title)


In [13]:
# for testing the crop_image function

# file_path = next(iter(list_ds))
# crop_image(file_path)

In [6]:
for file_path in iter(list_ds):
    crop_image(file_path)